<a href="https://colab.research.google.com/github/marinebon/HackingLimno2025/blob/main/02_Sample_environmental_data_at_each_occurrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [11]:
# Modify this section as needed.

# ==============================================================================
# === directory setup
# ==============================================================================
# If using google colab and google drive:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = '/content/drive/MyDrive/GSoC_SDM_Project'

# === if using local machine
# PROJECT_DIR = './'
#import os
#if not os.path.exists(PROJECT_DIR):
#    os.makedirs(PROJECT_DIR)

# ==============================================================================
# ==============================================================================
# === spatial coverage
# ==============================================================================
LATMIN = 24.11637699635014
LATMAX = 26.11949526731449
LONMIN = -82.51572158798965
LONMAX = -79.61106009492724
# ==============================================================================

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install rasterio pandas geopandas matplotlib  requests xarray netCDF4

mount google drive for saving result

In [13]:
# read parquet file of occurrences
# see step 01 https://github.com/marinebon/HackingLimno2025

import pandas as pd
%matplotlib inline
df_clean = pd.read_parquet(f'{PROJECT_DIR}/occurrences.parquet')

Function to prepare data for environmental sampling

In [14]:
from typing import Optional, List
import pandas as pd

# TODO: descriptions here
def prepare_for_environmental_sampling(
    df: pd.DataFrame,
    species_column: str = 'scientificName',
    lat_column: str = 'decimalLatitude',
    lon_column: str = 'decimalLongitude',
    additional_columns: Optional[List[str]] = None
) -> pd.DataFrame:

    columns = [species_column, lat_column, lon_column]

    if additional_columns:
        columns.extend([col for col in additional_columns if col in df.columns])

    prepared_df = df[columns].copy()

    prepared_df['occurrence_id'] = range(len(prepared_df))

    cols = ['occurrence_id'] + [col for col in prepared_df.columns if col != 'occurrence_id']
    prepared_df = prepared_df[cols]

    return prepared_df

bringing in environmental data. Functions for Environmental Data Sampling

In [15]:
import rasterio
import xarray as xr
import numpy as np
from typing import Union, List, Tuple, Optional, Dict

In [16]:
def add_environmental_data(
    df: pd.DataFrame,
    raster_path: str,
    column_name: str = 'temperature',
    lat_col: str = 'decimalLatitude',
    lon_col: str = 'decimalLongitude'
) -> pd.DataFrame:

    df_result = df.copy()

    # TODO: change to use xarray
    with rasterio.open(raster_path) as src:
        coords = [(row[lon_col], row[lat_col]) for _, row in df.iterrows()]
        sampled_values = list(src.sample(coords))
        band_index = 0  # 0 is surface
        values = [val[band_index] if val[band_index] != src.nodata else np.nan for val in sampled_values]
        df_result[column_name] = values

    print(f"Added {column_name}: {df_result[column_name].notna().sum()}/{len(df_result)} valid values")
    return df_result

# multidecadal environmental data from NOAA World Ocean Atlas
# https://www.ncei.noaa.gov/access/world-ocean-atlas-2023/

# TODO: what about depth?
df_with_env = add_environmental_data(df_clean, f'{PROJECT_DIR}/woa23_decav_t00_04_t_mn.tif', 'temperature')
df_with_env = add_environmental_data(df_with_env, f'{PROJECT_DIR}/woa23_decav_t00_04_t_sd.tif', 'temperature_sd')
# TODO: add more files here

print(df_with_env.head())

df_with_env.to_parquet(f'{PROJECT_DIR}/occurrences_and_environment.parquet', engine='pyarrow', index=False)

Added temperature: 570/3013 valid values
Added temperature_sd: 570/3013 valid values
   decimalLatitude  decimalLongitude occurrenceStatus  date_year  \
0        25.138383        -80.624983                Q     2006.0   
1        24.996280        -80.649250                Q     2006.0   
2        25.248783        -80.403150                Q     2005.0   
3        25.252020        -80.397290                Q     2007.0   
4        25.472200        -80.325950                Q     2005.0   

       date_mid  temperature  temperature_sd  
0  1.147133e+12          NaN             NaN  
1  1.160006e+12    26.382675        2.261415  
2  1.115683e+12    27.391775        1.604826  
3  1.189469e+12          NaN             NaN  
4  1.129680e+12          NaN             NaN  


In [19]:
# sample data using xarray and thredds
import xarray as xr

url = 'https://www.ncei.noaa.gov/thredds-ocean/fileServer/woa23/DATA/temperature/netcdf/decav/0.25/woa23_decav_t00_04.nc'

ds = xr.open_dataset(url, decode_times=False)  # lazy load

# TODO: for [ x for x in ]
ds['t_an'].sel(lat=24.5, lon=-81, depth=0, method='nearest').to_dataframe()

OSError: [Errno -90] NetCDF: file not found: 'https://www.ncei.noaa.gov/thredds-ocean/fileServer/woa23/DATA/temperature/netcdf/decav/0.25/woa23_decav_t00_04.nc'